### Загрузка


In [439]:
!gdown 1JFFUPMS9wWPCkBnhjBw9hJmINSk8V59T

Downloading...
From: https://drive.google.com/uc?id=1JFFUPMS9wWPCkBnhjBw9hJmINSk8V59T
To: /content/data.csv
100% 21.2M/21.2M [00:00<00:00, 186MB/s]


In [440]:
! pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [441]:
import pandas as pd
import numpy as np

In [442]:
from ipymarkup import show_span_ascii_markup as show_markup
import ipymarkup

from yargy import (
    Parser,
    rule, or_,
    not_, and_
)
from yargy.predicates import (
    eq, type, caseless, in_, in_caseless,
    gte, lte, length_eq,
    is_capitalized, normalized,
    dictionary, gram,
)
from yargy.pipelines import (
    caseless_pipeline,
    pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp
from yargy.relations import gnc_relation
from yargy.tokenizer import (
    QUOTES,
    LEFT_QUOTES,
    RIGHT_QUOTES,
    INT,
    LATIN,
    PUNCT,
    MorphTokenizer,
    TokenRule
)

from yargy import Parser
from yargy import rule
from yargy.predicates import eq
from yargy.interpretation import fact, attribute

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

morph_vocab = MorphVocab()

In [443]:
NOUN = type('NOUN')
COMMA = eq(',')
COLON = eq(':')
DOT = eq('.')
DASH = eq('-')
UNDSC = eq('_')
CACSES_L = eq('(')
CACSES_R = eq(')')

# Основной код

In [444]:
data = pd.read_csv('data.csv', index_col=0)
data

,Date,Message title,Message link,Message content,Company name,Company link
0,30.06.2020 19:37,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,"АО ""ЭЛИОН""",https://e-disclosure.ru/portal/company.aspx?id...
1,30.06.2020 19:36,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,"СООБЩЕНИЕ О СУЩЕСТВЕННОМ ФАКТЕ\n\n""СВЕДЕНИЯ О ...","ПАО ""МТС-Банк""",https://e-disclosure.ru/portal/company.aspx?id...
2,30.06.2020 19:35,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Сообщение о существенном факте о проведении об...,"АО ""Ситиматик""",https://e-disclosure.ru/portal/company.aspx?id...
3,30.06.2020 18:01,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,ПАО «Европейская Электротехника»,https://e-disclosure.ru/portal/company.aspx?id...
4,30.06.2020 17:33,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,"АО ""Борхиммаш""",https://e-disclosure.ru/portal/company.aspx?id...
...,...,...,...,...,...,...
1071,02.04.2020 11:49,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,"АО ""Молодая гвардия""",https://e-disclosure.ru/portal/company.aspx?id...
1072,02.04.2020 11:44,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,"ОАО ""Тулаавтотранс""",https://e-disclosure.ru/portal/company.aspx?id...
1073,01.04.2020 17:00,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Сообщение о существенном факте о проведении об...,"ПАО ""Группа Черкизово""",https://e-disclosure.ru/portal/company.aspx?id...
1074,01.04.2020 10:36,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...,Решения общих собраний участников (акционеров)...,"АО ""СЗ""",https://e-disclosure.ru/portal/company.aspx?id...


In [445]:
#удаляем пробельные символы
data.rename(columns={'Message content' : 'content'}, inplace=True)
data['content'].replace({'\r':' ', '\t':' ', '\n':' '}, regex=True, inplace=True)
data['content'].replace(r'\s+', ' ', regex=True, inplace=True)

In [446]:
showcase = pd.DataFrame(data['Company name'])
showcase.rename(columns={'Company name':'comp_name'}, inplace=True)
showcase

,comp_name
0,"АО ""ЭЛИОН"""
1,"ПАО ""МТС-Банк"""
2,"АО ""Ситиматик"""
3,ПАО «Европейская Электротехника»
4,"АО ""Борхиммаш"""
...,...
1071,"АО ""Молодая гвардия"""
1072,"ОАО ""Тулаавтотранс"""
1073,"ПАО ""Группа Черкизово"""
1074,"АО ""СЗ"""


#### ИНН Компании

In [447]:
#ищем подстроку с ИНН
import re 
def find_section(text):
  return re.findall(r'ИНН.+?([0-9]{10})', text, re.I | re.S)

In [448]:
prep_arr = []
for i in range(len(data.content)):
    text = data.iloc[i].content
    prep_arr.append(find_section(text))

In [449]:
def inn_ogrn_func(x): #берем первый найденный инн - инн компании
    if len(x) == 0:
        return np.NAN
    return x[0]

In [450]:
showcase['inn'] = prep_arr
showcase['inn'] = showcase['inn'].apply(func=inn_ogrn_func)
showcase

,comp_name,inn
0,"АО ""ЭЛИОН""",7735064331
1,"ПАО ""МТС-Банк""",7702045051
2,"АО ""Ситиматик""",7725727149
3,ПАО «Европейская Электротехника»,7716814300
4,"АО ""Борхиммаш""",3604001059
...,...,...
1071,"АО ""Молодая гвардия""",7707082177
1072,"ОАО ""Тулаавтотранс""",7107003984
1073,"ПАО ""Группа Черкизово""",7718560636
1074,"АО ""СЗ""",9106001900


#### ОГРН КОМПАНИИ

In [452]:
import re #ищем подстроку с ОГРН
def find_section(text):
  return re.findall(r'ОГРН.+?([0-9]{10})', text, re.I | re.S)

In [453]:
prep_arr = []
for i in range(len(data.content)):
    text = data.iloc[i].content
    prep_arr.append(find_section(text))


In [ ]:
prep_arr

[['1027700063', '1027700063'],
 ['1027739053', '7703097990'],
 ['1117746488'],
 ['1167746062', '7701017140'],
 ['1023600609'],
 ['1021300973', '1021300973'],
 ['1022303977', '6163132607'],
 ['1022303979', '6163132607'],
 ['1024300749'],
 ['1025901675'],
 ['1026104023'],
 ['1027700077'],
 ['1020201995'],
 ['1027700166', '7705051102', '7705051102', '7705051102', '1027700166'],
 ['1021801092'],
 ['1027739090'],
 ['1033302200'],
 ['1024800823'],
 ['1147746558', '1160305320'],
 ['1027801543'],
 ['1062646000'],
 ['1197456003'],
 ['1025202101', '1025202101'],
 ['1027400508'],
 ['1026301708'],
 ['1023404238'],
 ['1028600584'],
 ['1026000953'],
 ['1023600605'],
 ['1027700067', '1150603048', '1200602033'],
 ['1023800732'],
 ['1023802658'],
 ['1027700343'],
 ['1156451022'],
 ['1023405564'],
 ['1023302152'],
 ['1023101637'],
 ['1022101132'],
 ['1035004252'],
 ['1026801361'],
 ['1022305002'],
 ['1026600705'],
 ['1025003523'],
 ['1035005901'],
 ['1022700711'],
 ['1026605388'],
 ['1026601982'],
 ['10

In [454]:
showcase['ogrn'] = prep_arr
showcase['ogrn'] = showcase['ogrn'].apply(func=inn_ogrn_func)
showcase

,comp_name,inn,ogrn
0,"АО ""ЭЛИОН""",7735064331,1027700063
1,"ПАО ""МТС-Банк""",7702045051,1027739053
2,"АО ""Ситиматик""",7725727149,1117746488
3,ПАО «Европейская Электротехника»,7716814300,1167746062
4,"АО ""Борхиммаш""",3604001059,1023600609
...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318
1074,"АО ""СЗ""",9106001900,1149102072


#### Полное название

In [458]:
import re #ищем подстроку с названием
def find_section(text):
  return re.findall(r'Полное фирменное наименование эмитента(.+?)1.2', text, re.I | re.S)

In [459]:
full_arr = []
for i in range(len(data.content)):
    text = data.iloc[i].content
    full_arr.append(find_section(text))


In [460]:
#Выделяем ПОЛНОЕ НАЗВАНИЕ
CORP_TYPE = morph_pipeline([
                  'публичное',
                  'Закрытое',
                  'Открытое',
]).normalized

TYPE = morph_pipeline([
         'акционерное общество',
         'Общество с ограниченное ответственностью',
]).normalized

QUOTES1 = eq('„')


TYPES = rule(
    CORP_TYPE.optional(),
    TYPE.optional(),
    is_capitalized(),
    eq(LEFT_QUOTES).optional(),
    not_(eq(QUOTES)).repeatable(),
    eq(RIGHT_QUOTES).optional(),
)

parser = Parser(TYPES)

full1_arr = []
for i in range(len(full_arr)):
    f_arr = []
    for j in range(len(full_arr[i])):
        text = full_arr[i][j]
        arr = []
        for match in parser.findall(text):
            arr.append([_.value for _ in match.tokens])
        f_arr.append(arr)
    full1_arr.append(f_arr)
    

In [461]:
for i in range(len(full1_arr)):
    flat_list = [item for sublist in full1_arr[i][0] for item in sublist]
    full1_arr[i] = flat_list

In [475]:
def del_spaces(x): #удалим ненужные пробелы около ковычек
  return re.sub(r'["«]\s*([^"]*?)\s*["»]', r'"\1"', x)

In [476]:
showcase['full_name'] = full1_arr
showcase['full_name'] = showcase['full_name'].apply(' '.join)
showcase['full_name'] = showcase['full_name'].apply(del_spaces)
showcase

,comp_name,inn,ogrn,full_name
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН"""
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк"""
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами"""
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл..."
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр..."
...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия"""
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс"""
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово"""
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов..."


#### Сокращенное название

In [477]:
import re #ищем подстроку с сокращенным названием
def find_sections(text):
  return re.findall(r'Сокращ.*?нное фирменное .*?(.+?)1.3', text, re.I | re.S)

prep_arr = []
for i in range(len(data.content)):
    text = data.iloc[i].content
    prep_arr.append(find_sections(text))

In [478]:
#СОКРАЩЕННОЕ НАЗВАНИЕ
TYPES = rule(
    is_capitalized().repeatable(),
    eq(LEFT_QUOTES).optional(),
    not_(eq(QUOTES)).repeatable(),
    eq(RIGHT_QUOTES).optional(),
)

parser = Parser(TYPES)

full_arr = []
for i in range(len(prep_arr)):
    f_arr = []
    for j in range(len(prep_arr[i])):
        text = prep_arr[i][j]
        arr = []
        for match in parser.findall(text):
            arr.append([_.value for _ in match.tokens])
        f_arr.append(arr)
    full_arr.append(f_arr)
    

In [479]:
full_flat = []
for i in range(len(full_arr)):
    flat_list = [item for sublist in full_arr[i][0] for item in sublist]
    full_flat.append(flat_list)

In [480]:
showcase['short_name'] = full_flat
showcase['short_name'] = showcase['short_name'].apply(' '.join)
showcase['short_name'] = showcase['short_name'].apply(del_spaces)
showcase

,comp_name,inn,ogrn,full_name,short_name
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН"""
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк"""
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами"""
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника"""
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш"""
...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия"""
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс"""
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово"""
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ"""


##### АДРЕС

In [481]:
import re
def find_sections(text):
  return re.findall(r'1\.3.*?Место нахождени(.+?) 1\.4.?', text, re.I | re.S)

In [482]:
prep_arr = []
for i in range(len(data.content)):
    text = data.iloc[i].content
    prep_arr.append(find_sections(text))


In [483]:
prep_arr

[['я эмитента: 124460, г. Москва, г. Зеленоград, Панфиловский проспект,д.4',
  'я эмитента: 124460, г. Москва, Зеленоград, Панфиловский проспект, д. 4'],
 ['я эмитента 115432, г. Москва, проспект Андропова дом 18 корп.1'],
 ['я эмитента Российская Федерация, г. Москва'],
 ['я эмитента: 129344, г. Москва, улица Лётчика Бабушкина, дом 1, корп. 3, этаж 2, пом. IX, комн. 23'],
 ['я эмитента: 397164 Воронежская область, г.Борисоглебск, ул.Проходная, 4А'],
 ['я эмитента: Российская Федерация, Республика Мордовия, г. Саранск',
  'я эмитента: 430003, Республика Мордовия, г. Саранск, проспект Ленина, дом 50'],
 ['я эмитента: 353715, Краснодарский край, Каневской район, ст. Челбасская, ул.Красная, 119'],
 ['я эмитента: Российская Федерация, Краснодарский край, Каневской район, пос. Кубанская степь, ул. Набережная, 39'],
 ['я эмитента: 613047, Кировская область, город Кирово-Чепецк, улица Ленина, д.1 корп.Б'],
 ['я эмитента: 618320, г. Александровск, Пермский край, ул. Войкова, 3'],
 ['я эмитента

In [484]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

addr_extractor = AddrExtractor(morph_vocab) #используем встроенный экстрактор для адресов

In [485]:
all_matches = [] #применяем экстрактор
for i in range(len(prep_arr)):
    line_matches = []
    matches_len = []
    for line in prep_arr[i]:
        if addr_extractor.find(line) == None:
          continue
        line_match = addr_extractor.find(line).fact.parts
        lines = []
        for part in line_match:
            lines.append(str(part.type) + ' ' + str(part.value))
            lines.append(',')
        line_matches.append(lines[:-1])
        matches_len.append(len(lines))
    if len(matches_len) == 0:
          all_matches.append(['Россия'])
          continue
    all_matches.append(line_matches[matches_len.index(max(matches_len))])
        

In [ ]:
all_matches

In [515]:
def del_spaces2(x):
    return x.replace(" ,",",")

In [516]:
showcase['addr'] = all_matches
showcase['addr'] = showcase['addr'].apply(' '.join)
showcase['addr'] = showcase['addr'].apply(del_spaces2)
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form,date,dividents,audit_name,audit_inn,dir_q
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460, город Москва, город Зеленоград,...",заочное голосование,30/06/2020,Принято решение выплачивать дивиденды,Финансовые и бухгалтерские консультанты,7701017140,Вопрос не поднимался
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432, город Москва, проспект Андропов...",заочное голосование,29/06/2020,,МТС-Банк,7703097990,Вопрос поднимался
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование,30/06/2020,,Аудит и право,,Вопрос поднимался
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344, город Москва, улица Лётчика Баб...",заочного голосования,26/06/2020,,Европейская Электротехника,7701017140,Вопрос поднимался
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164, область Воронежская, город Бори...",заочное,30/06/2020,,,,Вопрос не поднимался
...,...,...,...,...,...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015, город Москва, дом 5А, строение ...",присутствие,02/04/2020,,"Общество с ограниченной ответственностью ""Инт...",,Вопрос не поднимался
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600, страна РФ, город Тула, улица Ту...",собрание,26/03/2020,,Аудит-Сервис,,Вопрос поднимался
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская, город Кашира, деревня Топк...",присутствие,01/04/2020,,Делойт и Туш СНГ,,Вопрос поднимался
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002, республика Крым, город Краснопе...",присутствие,27/03/2020,,,,Вопрос не поднимался


##### ФОРМА СОБРАНИЯ

In [488]:
import string

In [489]:
#применяем yargy парсер
TYPES = morph_pipeline([
                      'заочное',
                      'очное',
                      'очное голосование',
                      'заочное голосование',
                      'присутствие',
]).normalized

G = morph_pipeline({
                      'собрание',
                      'Собрание'
}).normalized

FORM = rule(
    or_(
        rule(
            TYPES,
        ),
        rule(
            COLON,
            G
        ),
        rule(
            DASH,
            G
        ),
        rule(
            eq('–'),
             G
        ),
        rule(
            DOT,
            G
        )
    )
).normalized

parser = Parser(FORM)
prep_arr = data.content

full_arr = []
for i in range(len(prep_arr)):
        text = prep_arr[i]
        arr = []
        for match in parser.findall(text):
            arr.append([_.value for _ in match.tokens])
        full_arr.append(arr)

In [490]:
full_arr

[[['заочное', 'голосование']],
 [['заочное', 'голосование']],
 [[':', 'собрание'], ['заочное', 'голосование']],
 [['присутствие'],
  ['заочное', 'голосование'],
  ['заочное', 'голосование'],
  ['заочного', 'голосования']],
 [['Заочное']],
 [['заочное', 'голосование']],
 [['заочное', 'голосование']],
 [[':', 'собрание'], ['заочное', 'голосование']],
 [['заочное', 'голосование']],
 [['присутствие'],
  ['заочное', 'голосование'],
  ['Заочное', 'голосование'],
  ['заочного', 'голосования']],
 [['ЗАОЧНОЕ', 'ГОЛОСОВАНИЕ'], ['заочного', 'голосования']],
 [['Заочное', 'голосование']],
 [['заочное', 'голосование']],
 [['присутствие'],
  ['заочное', 'голосование'],
  [':', 'собрание'],
  ['присутствие']],
 [[':', 'собрание']],
 [['заочного', 'голосования']],
 [['заочное', 'голосование']],
 [['заочное', 'голосование']],
 [['присутствие'], ['заочное', 'голосование'], ['заочное', 'голосование']],
 [['заочное', 'голосование']],
 [[':', 'собрание'], ['присутствие']],
 [['присутствие'],
  ['заочное', 

In [491]:
for i in range(len(full_arr)):
    if len(full_arr[i]) > 0:
        full_arr[i] = full_arr[i][-1]

In [492]:
def list_to_str(x):
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [s for s in x if s]
    return ' '.join(x).lower()

In [493]:
showcase['form'] = full_arr
showcase['form'] = showcase['form'].apply(list_to_str)
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460 , город Москва , город Зеленогра...",заочное голосование
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432 , город Москва , проспект Андроп...",заочное голосование
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344 , город Москва , улица Лётчика Б...",заочного голосования
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164 , область Воронежская , город Бо...",заочное
...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015 , город Москва , дом 5А , строен...",присутствие
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600 , страна РФ , город Тула , улица...",собрание
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская , город Кашира , деревня То...",присутствие
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002 , республика Крым , город Красно...",присутствие


In [543]:
showcase.replace({'form': {'заочная': 'заочное', 'заочного': 'заочное', 'заочной':'заочное', 'заочном':'заочное',
                           'заочного голосование':'заочное голосование', 'заочного голосования':'заочное голосование', 'заочное голосования':'заочное голосование',
                           'очная':'очное', 'присутствие':'очное','присутствия':'очное','собрания':'собрание', '– собрание':'собрание'}}, inplace=True)

##### ДАТА СОБРАНИЯ

In [494]:
dates_extractor = DatesExtractor(morph_vocab) #будем использовать встроенный экстрактор для дат

In [496]:
def date_func(string): #ищем нужную подстроку, вырезаем ее, применяем экстрактор
    re_list = ['[Дд]ата\sпроведения', '[Дд]ата\sпроведения\s', '[Дд]ата (момент) наступления',
           '[Дд]ата\sнаступления',
           '[Дд]ата,\sместо', '[Дд]ата\sи\sместо', '[Дд]ата (момент) наступления',
           '[Дд]ата\sи\sпочтовый\sадрес']

    pos_list = []

    for line in re_list:
        pos = re.search(line, string)
        pos_list.append(pos)

    fin_pos = None
    for i in pos_list:
        if i != None:
            fin_pos = i
            break;
    if fin_pos != None:
        tmp = string[fin_pos.end():fin_pos.end() + 250]
        tmp = re.sub('[' + re.escape(QUOTES) + ']', '', tmp)
        match = dates_extractor.find(tmp)
        if match.fact.month == None or match.fact.month == None:
            tmp = string[fin_pos.end() + match.stop:fin_pos.end() + 250]
            match = dates_extractor.find(tmp)

        month = match.fact.month if match.fact.month > 9 else '0' + str(match.fact.month)
        day = match.fact.day if match.fact.day > 9 else '0' + str(match.fact.day)
        date = [str(day), str(month), str(match.fact.year)]
        return '/'.join(date)
    return np.NAN

In [497]:
all_dates = []
for i in range(len(data.content)):
    all_dates.append(date_func(data.content[i]))

In [498]:
all_dates

['30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '26/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '29/06/2020',
 '26/06/2020',
 '30/06/2020',
 '30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '26/06/2020',
 '30/06/2020',
 '25/06/2020',
 '29/06/2020',
 '26/06/2020',
 '30/06/2020',
 '30/06/2020',
 '26/06/2020',
 '30/06/2020',
 '30/06/2020',
 '26/06/2020',
 '26/06/2020',
 '30/06/2020',
 '26/06/2020',
 '25/06/2020',
 '30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '30/06/2020',
 '30/06/2020',
 '29/06/2020',
 '30/06/2020',
 '30/06/2020',
 '26/06/2020',
 '30/06/2020',
 '30/06/2020',
 '26/06/2020',
 '29/06/2020',
 '26/06/2020',
 '30/06/2020',
 '26/06/2020',
 '27/06/2020',
 '29/06/2020',
 '30/06/2020',
 '25/06/2020',
 '26/06/2020',
 '26/06/2020',
 '30/06/2020',
 '24/06/2020',
 '27/06/2020',
 '26/06/2020',
 '30/06/2020',
 '26/06/2020',
 '26/06/20

In [499]:
showcase['date'] = all_dates
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form,date
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460 , город Москва , город Зеленогра...",заочное голосование,30/06/2020
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432 , город Москва , проспект Андроп...",заочное голосование,29/06/2020
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование,30/06/2020
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344 , город Москва , улица Лётчика Б...",заочного голосования,26/06/2020
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164 , область Воронежская , город Бо...",заочное,30/06/2020
...,...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015 , город Москва , дом 5А , строен...",присутствие,02/04/2020
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600 , страна РФ , город Тула , улица...",собрание,26/03/2020
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская , город Кашира , деревня То...",присутствие,01/04/2020
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002 , республика Крым , город Красно...",присутствие,27/03/2020


##### ДИВИДЕНТЫ

In [526]:
showcase['dividents'] = ''
messages = list(pd.read_csv('data.csv').drop(columns=list(data)[0])['Message content'])

In [527]:
for i in range(len(messages)):
    text = messages[i]
    start_list = [n.span() for n in re.finditer('ивиденд', text)]
    if start_list != []:
        showcase['dividents'][i] = 'Принято решение выплачивать дивиденды'
        for start_ind in start_list:
            start = start_ind[0]
            end = list(re.finditer('\t|\n|\r', text[start:]))[0].span()[0]
            dividents_info = text[start:start+end]
            if dividents_info.find('не выпла') + \
                dividents_info.find('не объявл') + \
                dividents_info.find('не начисл') + \
                text.find('е объявлять дивиденды') + \
                text.find('е выплачивать (объявлять) дивиденды') + \
                text.find('е выплачивать дивиденды') != -6:
                showcase['dividents'][i] = 'Принято решение не выплачивать дивиденды'
                break
    else:
        showcase['dividents'][i] = 'Вопрос не поднимался'

In [528]:
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form,date,dividents,audit_name,audit_inn,dir_q
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460, город Москва, город Зеленоград,...",заочное голосование,30/06/2020,Принято решение не выплачивать дивиденды,Финансовые и бухгалтерские консультанты,7701017140,Вопрос не поднимался
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432, город Москва, проспект Андропов...",заочное голосование,29/06/2020,Принято решение не выплачивать дивиденды,МТС-Банк,7703097990,Вопрос поднимался
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование,30/06/2020,Принято решение не выплачивать дивиденды,Аудит и право,,Вопрос поднимался
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344, город Москва, улица Лётчика Баб...",заочного голосования,26/06/2020,Принято решение не выплачивать дивиденды,Европейская Электротехника,7701017140,Вопрос поднимался
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164, область Воронежская, город Бори...",заочное,30/06/2020,Принято решение выплачивать дивиденды,,,Вопрос не поднимался
...,...,...,...,...,...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015, город Москва, дом 5А, строение ...",присутствие,02/04/2020,Принято решение не выплачивать дивиденды,"Общество с ограниченной ответственностью ""Инт...",,Вопрос не поднимался
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600, страна РФ, город Тула, улица Ту...",собрание,26/03/2020,Принято решение не выплачивать дивиденды,Аудит-Сервис,,Вопрос поднимался
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская, город Кашира, деревня Топк...",присутствие,01/04/2020,Принято решение выплачивать дивиденды,Делойт и Туш СНГ,,Вопрос поднимался
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002, республика Крым, город Краснопе...",присутствие,27/03/2020,Вопрос не поднимался,,,Вопрос не поднимался


#### АУДИТОР

In [507]:
import re 
def find_section(text):
  return re.findall(r'ИНН.+?([0-9]{10})', text, re.I | re.S)

In [508]:
def find_audit(text):
    match = re.search(r"Утвердить [а-я\s]*аудитором", text)
    if match == None:
        match = re.search(r"в качестве [а-я\s]*аудитора", text)
        if match == None:
            return ['', '']
    start = match.end()
    audit = ''
    audit_name = ''
    audit_inn = ''
    if start != -2:
        audit_text = text[start:]
        audit_name = audit_text[audit_text.find('«')+1:audit_text.find('»')]
        audit_inn = ''
        if audit_text.find('ИНН') != -1:
            audit_inn = find_section(audit_text)
            if len(audit_inn) == 0:
                return [audit_name, '']
            return [audit_name, audit_inn[0]]
    return [audit_name, audit_inn]

In [509]:
audit_inn = [''] * len(data.content)
audit_name = [''] * len(data.content)
for i in range(len(data.content)):
    text = data.content[i]
    audit_name[i], audit_inn[i] = find_audit(text)

In [ ]:
audit_inn

In [510]:
showcase['audit_name'] = audit_name
showcase['audit_inn'] = audit_inn
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form,date,dividents,audit_name,audit_inn
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460 , город Москва , город Зеленогра...",заочное голосование,30/06/2020,Принято решение выплачивать дивиденды,Финансовые и бухгалтерские консультанты,7701017140
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432 , город Москва , проспект Андроп...",заочное голосование,29/06/2020,,МТС-Банк,7703097990
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование,30/06/2020,,Аудит и право,
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344 , город Москва , улица Лётчика Б...",заочного голосования,26/06/2020,,Европейская Электротехника,7701017140
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164 , область Воронежская , город Бо...",заочное,30/06/2020,,,
...,...,...,...,...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015 , город Москва , дом 5А , строен...",присутствие,02/04/2020,,"Общество с ограниченной ответственностью ""Инт...",
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600 , страна РФ , город Тула , улица...",собрание,26/03/2020,,Аудит-Сервис,
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская , город Кашира , деревня То...",присутствие,01/04/2020,,Делойт и Туш СНГ,
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002 , республика Крым , город Красно...",присутствие,27/03/2020,,,


#### Совет директоров

In [511]:
def dir_func(text):
    re_list = ['[Ии]збра(.*?)[Сс]овет* [Дд]иректоров', '[Ии]збра(.*?)[Нн]аблюдательн*[Сс]овета']

    pos_list = []

    for line in re_list:
        pos = re.search(line, text)
        pos_list.append(pos)
    if pos_list[0] == None and pos_list[1] == None:
        return 'Вопрос не поднимался'
    return 'Вопрос поднимался'

In [512]:
dir_q = [''] * len(data.content)
for i in range(len(data.content)):
    text = data.content[i]
    dir_q[i] = dir_func(text)

In [513]:
showcase['dir_q'] = dir_q
showcase

,comp_name,inn,ogrn,full_name,short_name,addr,form,date,dividents,audit_name,audit_inn,dir_q
0,"АО ""ЭЛИОН""",7735064331,1027700063,"Акционерное общество ""ЭЛИОН""","АО ""ЭЛИОН""","индекс 124460 , город Москва , город Зеленогра...",заочное голосование,30/06/2020,Принято решение выплачивать дивиденды,Финансовые и бухгалтерские консультанты,7701017140,Вопрос не поднимался
1,"ПАО ""МТС-Банк""",7702045051,1027739053,"Публичное акционерное общество ""МТС - Банк""","ПАО ""МТС - Банк""","индекс 115432 , город Москва , проспект Андроп...",заочное голосование,29/06/2020,,МТС-Банк,7703097990,Вопрос поднимался
2,"АО ""Ситиматик""",7725727149,1117746488,"Акционерное общество ""Управление отходами""","АО ""Управление отходами""",город Москва,заочное голосование,30/06/2020,,Аудит и право,,Вопрос поднимался
3,ПАО «Европейская Электротехника»,7716814300,1167746062,"Публичное акционерное общество ""Европейская Эл...","ПАО ""Европейская Электротехника""","индекс 129344 , город Москва , улица Лётчика Б...",заочного голосования,26/06/2020,,Европейская Электротехника,7701017140,Вопрос поднимался
4,"АО ""Борхиммаш""",3604001059,1023600609,"Акционерное общество ""Борисоглебский ордена Тр...","АО ""Борхиммаш""","индекс 397164 , область Воронежская , город Бо...",заочное,30/06/2020,,,,Вопрос не поднимался
...,...,...,...,...,...,...,...,...,...,...,...,...
1071,"АО ""Молодая гвардия""",7707082177,1027739804,"Акционерное общество ""Молодая гвардия""","АО ""Молодая гвардия""","индекс 127015 , город Москва , дом 5А , строен...",присутствие,02/04/2020,,"Общество с ограниченной ответственностью ""Инт...",,Вопрос не поднимался
1072,"ОАО ""Тулаавтотранс""",7107003984,1027100966,"Открытое акционерное общество ""Тулаавтотранс""","ОАО ""Тулаавтотранс""","индекс 300600 , страна РФ , город Тула , улица...",собрание,26/03/2020,,Аудит-Сервис,,Вопрос поднимался
1073,"ПАО ""Группа Черкизово""",7718560636,1057748318,"Публичное акционерное общество ""Группа Черкизово""","ПАО ""Группа Черкизово""","область Московская , город Кашира , деревня То...",присутствие,01/04/2020,,Делойт и Туш СНГ,,Вопрос поднимался
1074,"АО ""СЗ""",9106001900,1149102072,"Публичное акционерное общество ""Крымский содов...","ПАО ""СЗ""","индекс 296002 , республика Крым , город Красно...",присутствие,27/03/2020,,,,Вопрос не поднимался


In [ ]:
dir_q

In [529]:
showcase.to_csv('final_showcase.csv', sep=';', index=False)